# Bibliotecas utilizadas

In [37]:
import pandas as pd

#preprocessamento
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# seleção do modelo
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV


#modelos
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor


# Importando os dados

In [38]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv('test.csv')

# Recaptulando os dados 
De treinamento e vendo o de teste

In [39]:
df_train

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969
...,...,...,...,...,...,...,...,...,...,...,...,...
550063,1006033,P00372445,M,51-55,13,B,1,1,20,NaN,NaN,368
550064,1006035,P00375436,F,26-35,1,C,3,0,20,NaN,NaN,371
550065,1006036,P00375436,F,26-35,15,B,4+,1,20,NaN,NaN,137
550066,1006038,P00375436,F,55+,1,C,2,0,20,NaN,NaN,365


In [40]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   User_ID                     550068 non-null  int64  
 1   Product_ID                  550068 non-null  object 
 2   Gender                      550068 non-null  object 
 3   Age                         550068 non-null  object 
 4   Occupation                  550068 non-null  int64  
 5   City_Category               550068 non-null  object 
 6   Stay_In_Current_City_Years  550068 non-null  object 
 7   Marital_Status              550068 non-null  int64  
 8   Product_Category_1          550068 non-null  int64  
 9   Product_Category_2          376430 non-null  float64
 10  Product_Category_3          166821 non-null  float64
 11  Purchase                    550068 non-null  int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 50.4+ MB


In [41]:
df_train['Product_Category_1'].value_counts().sort_index()

Product_Category_1
1     140378
2      23864
3      20213
4      11753
5     150933
6      20466
7       3721
8     113925
9        410
10      5125
11     24287
12      3947
13      5549
14      1523
15      6290
16      9828
17       578
18      3125
19      1603
20      2550
Name: count, dtype: int64

In [42]:
df_test

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,NaN
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,NaN
2,1000010,P00288442,F,36-45,1,B,4+,1,5,14.0,NaN
3,1000010,P00145342,F,36-45,1,B,4+,1,4,9.0,NaN
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...
233594,1006036,P00118942,F,26-35,15,B,4+,1,8,NaN,NaN
233595,1006036,P00254642,F,26-35,15,B,4+,1,5,8.0,NaN
233596,1006036,P00031842,F,26-35,15,B,4+,1,1,5.0,12.0
233597,1006037,P00124742,F,46-50,1,C,4+,0,10,16.0,NaN


In [43]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233599 entries, 0 to 233598
Data columns (total 11 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   User_ID                     233599 non-null  int64  
 1   Product_ID                  233599 non-null  object 
 2   Gender                      233599 non-null  object 
 3   Age                         233599 non-null  object 
 4   Occupation                  233599 non-null  int64  
 5   City_Category               233599 non-null  object 
 6   Stay_In_Current_City_Years  233599 non-null  object 
 7   Marital_Status              233599 non-null  int64  
 8   Product_Category_1          233599 non-null  int64  
 9   Product_Category_2          161255 non-null  float64
 10  Product_Category_3          71037 non-null   float64
dtypes: float64(2), int64(4), object(5)
memory usage: 19.6+ MB


In [44]:
df_test['Product_Category_1'].value_counts().sort_index()

Product_Category_1
1     60321
2     10192
3      8578
4      5003
5     65017
6      8860
7      1624
8     48369
9       194
10     2248
11    10153
12     1663
13     2381
14      663
15     2694
16     4105
17      223
18     1311
Name: count, dtype: int64

Percebemos que os dados do teste não apresentam os outliers que encontramos no de treinamento (categoria 19 e 20) que vimos quando fizemos a EDA.
Porem apresenta a mesma questão de dados faltantes no Product_Category_2 e 3.

# Tratamento

Iremos começar o tratamento, buscando lidar com os dados faltantes, Outliers e tratar os dados categorigos. 

In [45]:
df_train.drop(df_train[(df_train.Product_Category_1 == 19) | (df_train.Product_Category_1 == 20)].index,inplace=True)

In [46]:
df_train['Product_Category_1'].value_counts().sort_index()


Product_Category_1
1     140378
2      23864
3      20213
4      11753
5     150933
6      20466
7       3721
8     113925
9        410
10      5125
11     24287
12      3947
13      5549
14      1523
15      6290
16      9828
17       578
18      3125
Name: count, dtype: int64

Veremos a quantidade e a porcentagem de valores nulos perante os dados de treinamento e de teste das colunas: Product_category_2 e 3.

In [47]:
def nulos_e_porcentagem(df):
    total = df.iloc[:, 9:11].copy().isnull().sum()
    porcentagem = df.iloc[:, 9:11].isnull().sum()/df.iloc[:, 9:11].isnull().count().sort_values(ascending=False)
    dados_nulos = pd.concat([total, porcentagem], axis=1, sort=False, keys=['total', 'porcentagem'])
    dados_nulos = dados_nulos[dados_nulos['porcentagem']!=0]*100
    return dados_nulos

In [48]:
train = nulos_e_porcentagem(df_train)
train

,total,porcentagem
Product_Category_2,16948500,31.046042
Product_Category_3,37909400,69.441946


In [49]:
test = nulos_e_porcentagem(df_test)
test

,total,porcentagem
Product_Category_2,7234400,30.969311
Product_Category_3,16256200,69.590195


Notasse que a porcentagem perante o dados de treinamento e teste, apresentam uma valor aproximado.
E percebe-se que O product_Category_3 tem uma quantia majoritaria de valores nulos, logo que sabemos que os produtos 
podem ter mutiplas categorias, logo iremos apagar a coluna Product_Category_3 e substituir os valores faltantes do Product_Category_2.

In [50]:
def apagando_e_substituindo(df):
    df.drop('Product_Category_3', axis=1, inplace=True)
    df['Product_Category_2'] = df['Product_Category_2'].fillna(-1).astype(int)

apagando_e_substituindo(df_train)
apagando_e_substituindo(df_test)


Lidaremos com a coluna <age> que se encontra como object os substituindos pela media.

In [51]:
media_idade = {'0-17':17, '18-25':21, '26-35':30, '36-45':40, '46-50':48, '51-55':53, '55+':60}
df_train["Age"] = df_train["Age"].apply(lambda x: media_idade[x])
df_test["Age"] = df_test["Age"].apply(lambda x: media_idade[x])

In [52]:
df_train['Stay_In_Current_City_Years'].replace('4+',5, inplace=True)

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_16296\974422290.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train['Stay_In_Current_City_Years'].replace('4+',5, inplace=True)


In [53]:
df_train.head(20)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Purchase
0,1000001,P00069042,F,17,10,A,2,0,3,-1,8370
1,1000001,P00248942,F,17,10,A,2,0,1,6,15200
2,1000001,P00087842,F,17,10,A,2,0,12,-1,1422
3,1000001,P00085442,F,17,10,A,2,0,12,14,1057
4,1000002,P00285442,M,60,16,C,5,0,8,-1,7969
5,1000003,P00193542,M,30,15,A,3,0,1,2,15227
6,1000004,P00184942,M,48,7,B,2,1,1,8,19215
7,1000004,P00346142,M,48,7,B,2,1,1,15,15854
8,1000004,P0097242,M,48,7,B,2,1,1,16,15686
9,1000005,P00274942,M,30,20,A,1,1,8,-1,7871


In [ ]:
df_train['Product_Count'] = df_train.Product_ID.groupby(df_train.Product_ID).transform('count')


In [61]:
df_train = pd.get_dummies(df_train, columns=['Gender','City_Category'], drop_first=True)